## Wikipedia Article Retrieval Tool

This section defines a simple Python function, `get_article`, that uses the `wikipedia` Python package to search for and retrieve the content of a Wikipedia article based on a search term. This function will be used as a tool for the agent to fetch up-to-date information from Wikipedia.

In [3]:
import wikipedia

def get_article(search_term):
    results = wikipedia.search(search_term)
    first_result = results[0]
    page = wikipedia.page(first_result, auto_suggest=False)
    return page.content

## Example: Fetching and Previewing Wikipedia Articles

Here, we demonstrate how to use the `get_article` function to retrieve and preview the content of Wikipedia articles for various search terms, such as "Avengers: Doomsday", "Nezha 2", "History of Malaysia", and "Iron Man". Only a preview of the article content is printed for brevity.

In [4]:
article = get_article("Avengers: Doomsday")
print(article[:1000]) # article is very long, so let's just print a preview

Avengers: Doomsday is an upcoming American superhero film based on the Marvel Comics superhero team the Avengers. Produced by Marvel Studios and AGBO, and distributed by Walt Disney Studios Motion Pictures, it is intended to be the fifth installment in the Avengers film series following Avengers: Endgame (2019) and the 39th film in the Marvel Cinematic Universe (MCU). Directed by Anthony and Joe Russo and written by Michael Waldron and Stephen McFeely, the film features an ensemble cast including Chris Hemsworth, Vanessa Kirby, Anthony Mackie, Sebastian Stan, Letitia Wright, Paul Rudd, Wyatt Russell, Tenoch Huerta Mejía, Ebon Moss-Bachrach, Simu Liu, Florence Pugh, Kelsey Grammer, Lewis Pullman, Danny Ramirez, Joseph Quinn, David Harbour, Winston Duke, Hannah John-Kamen, Tom Hiddleston, Patrick Stewart, Ian McKellen, Alan Cumming, Rebecca Romijn, James Marsden, Channing Tatum, Pedro Pascal, and Robert Downey Jr.. In the film, the Avengers, Wakandans, Fantastic Four, New Avengers, and t

In [5]:
article = get_article("Nezha 2")
print(article[:500]) # article is very long, so let's just print a preview

Ne Zha 2 (Chinese: 哪吒之魔童闹海; pinyin: Nézhā zhī Mótóng nào hǎi; also known as 哪吒2; Nézhā èr) is a 2025 Chinese animated adventure comedy film written and directed by Jiaozi. The direct sequel to Ne Zha (2019), it is based on the Chinese mythological character and Xu Zhonglin's 16th-century novel Investiture of the Gods (Fengshen Yanyi). The film takes up the story of Chinese mythological character Ne Zha and his friend Ao Bing. After a sacrifice, only Ne Zha’s body can be recreated, although he ca


In [6]:
article = get_article("History of Malaysia")
print(article[:3000]) #article is super long so let's just print a preview

Malaysia is a modern concept, created in the second half of the 20th century. However, contemporary Malaysia regards the entire history of Malaya and Borneo, spanning thousands of years back to prehistoric times, as its own history. Significant events in Malaysia's modern history include the formation of the federation, the separation of Singapore, the racial riots, Mahathir Mohamad's era of industrialisation and privatisation, and the nation's political upheavals of the late 20th and early 21st centuries.
The first evidence of archaic human occupation in the region dates back at least 1.83 million years, while the earliest remnants of anatomically modern humans are approximately 40,000 years old. The ancestors of the present-day population of Malaysia entered the area in multiple waves during prehistoric and historical times.
Hinduism and Buddhism from India and China dominated early regional history, reaching their peak from the 7th to the 13th centuries during the reign of the Sumat

In [7]:
article = get_article("Iron Man")
print(article[:1000]) #article is super long so let's just print a preview

Iron Man is a superhero appearing in American comic books published by Marvel Comics. Co-created by writer and editor Stan Lee, developed by scripter Larry Lieber, and designed by artists Don Heck and Jack Kirby, the character first appeared in Tales of Suspense #39 in 1962 (cover dated March 1963) and received his own title with Iron Man #1 in 1968. Shortly after his creation, Iron Man became a founding member of the superhero team, the Avengers, alongside Thor, Ant-Man, the Wasp, and the Hulk. Iron Man stories, individually and with the Avengers, have been published consistently since the character's creation.
Iron Man is the superhero persona of Anthony Edward "Tony" Stark, a businessman and engineer who runs the weapons manufacturing company Stark Industries. When Stark was captured in a war zone and sustained a severe heart wound, he built his Iron Man armor and escaped his captors. Iron Man's suits of armor grant him superhuman strength, flight, energy projection, and other abili

## Tool Schema Definition for Agent Use

This cell defines a tool schema dictionary, `article_search_tool`, which describes the Wikipedia retrieval tool in a format compatible with agentic LLM APIs (like Anthropic Claude). The schema includes the tool's name, description, input schema, and required fields.

In [8]:
article_search_tool = {
    "name": "get_article",
    "description": "A tool to retrieve an up to date Wikipedia article.",
    "input_schema": {
        "type": "object",
        "properties": {
            "search_term": {
                "type": "string",
                "description": "The search term to find a wikipedia article by title"
            },
        },
        "required": ["search_term"]
    }
}

## Setting Up Anthropic Client and Making a Tool-Use Request

This section initializes the Anthropic API client and demonstrates how to send a message to the Claude model, asking a question that may require tool use (e.g., "What is the box office for Nezha 2?"). The tool schema is provided to the model, enabling it to call the Wikipedia tool if needed.

In [11]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()

client = Anthropic()

messages = [{"role": "user", "content": "What is the box office for Nezha 2"}]

response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        messages=messages,
        max_tokens=1000,
        tools=[article_search_tool]
    )

TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

In [ ]:
response.content

## Appending the Assistant's Tool Use to the Conversation

This cell appends the assistant's tool use response to the ongoing conversation history, preparing for the next step in the agentic interaction.

In [ ]:
messages.append({"role": "assistant", "content": response.content})

## Extracting Tool Use Information

This section extracts the tool name and input parameters from the model's tool use response, so that the tool can be called programmatically.

In [ ]:
# This is a simple, but brittle way of getting the tool use information
# We're simply taking the last block from Claude's response.
tool_use = response.content[-1]
tool_name = tool_use.name
tool_input = tool_use.input
print("Tool name: ", tool_name)
print("Tool input", tool_input)

## Executing the Tool and Returning Results

Here, the code checks if the model requested the Wikipedia tool, executes the tool with the provided input, and prints a preview of the Wikipedia article content.

In [ ]:
if tool_name == "get_article":
    search_term = tool_input["search_term"]
    wiki_result = get_article(search_term)
    print(f"Searching Wikipedia for {search_term}")
    print("WIKIPEDIA PAGE CONTENT:")
    print(wiki_result[:500]) #just printing a small bit of the article because it's so long

## Constructing a Tool Result Message

This cell shows how to construct a message containing the tool result, formatted according to the agentic API's expectations, so it can be sent back to the model.

In [ ]:
{
  "role": "user",
  "content": [
    {
      "type": "tool_result",
      "tool_use_id": "toolu_01A09q90qw90lq917835lq9",
      "content": "The result of actually calling the tool goes here"
    }
  ]
}

## Sending the Tool Result to the Conversation

The tool result message is appended to the conversation history, allowing the agent to use the tool's output in its next response.

In [ ]:
tool_response = {
    "role": "user",
    "content": [
        {
        "type": "tool_result",
        "tool_use_id": tool_use.id,
        "content": wiki_result
        }
    ]
}

In [ ]:
tool_response

In [ ]:
messages.append(tool_response)

In [ ]:
follow_up_response = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    messages=messages,
    max_tokens=1000,
    tools=[article_search_tool]
)

In [ ]:
follow_up_response.content[0].text

## Model Follow-up: Final Answer Generation

This section sends the updated conversation (including the tool result) back to the model, prompting it to generate a final answer that incorporates the information retrieved from Wikipedia.

In [ ]:

def answer_question(question):
    messages = [{"role": "user", "content": question}]

    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        messages=messages,
        max_tokens=1000,
        tools=[article_search_tool]
    )
    
    if(response.stop_reason == "tool_use"):
        tool_use = response.content[-1]
        tool_name = tool_use.name
        tool_input = tool_use.input
        #Add Claude's tool use call to messages:
        messages.append({"role": "assistant", "content": response.content})

        if tool_name == "get_article":
            search_term = tool_input["search_term"]
            print(f"Claude wants to get an article for {search_term}")
            wiki_result = get_article(search_term) #get wikipedia article content
            #construct our tool_result message
            tool_response = {
                "role": "user",
                "content": [
                    {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,
                    "content": wiki_result
                    }
                ]
                }
            messages.append(tool_response)
            #respond back to Claude
            response = client.messages.create(
                model="claude-3-7-sonnet-20250219",
                messages=messages,
                max_tokens=1000,
                tools=[article_search_tool]
            )
            print("Claude's final answer:")
            print(response.content[0].text)

    else:
        print("Claude did not call our tool")
        print(response.content[0].text)

## Agentic Tool Use: Full Question-Answering Loop

This cell defines a reusable function, `answer_question`, that demonstrates the full agentic tool-use loop: sending a question to the model, detecting tool use, executing the tool, sending the result, and printing the model's final answer.

In [ ]:
answer_question("What are the names of all the Avengers films in the Marvel Cinematic Universe?")